In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

Создаем пример. Я люблю авиацию, поэтому представлю, что работаю в авиакомпании, и мне нужно построить моодель линейной регрессии, предсказав стоимость керосина (amount) , который мы купим и используем для перевозок в следующем месяце. Допустим, у нас есть датасет с признаками: 

* mounth - месяц перевозки
* price_kerosin - цена на керосин по перевозке
* volume - сколько литров потрачено

In [8]:
n_samples = 1000

In [16]:
mounth = np.random.choice(12, n_samples) + 1
price_kerosin = np.random.choice(100, n_samples) + 38
volume = np.random.choice(27500, n_samples) + 4500

# при условии, что 48 литров самолёт расходует в час, и есть еще некий сдвиг из-за ветра, получим:

amount = price_kerosin * volume * 48 + 1.7

df = pd.DataFrame({'mounth': mounth, 'price_kerosin': price_kerosin, 'volume': volume, 'amount': amount})
df.head()

,mounth,price_kerosin,volume,amount
0,10,55,10943,28889521.7
1,11,132,10951,69385537.7
2,7,43,21352,44070529.7
3,10,41,6151,12105169.7
4,11,50,20971,50330401.7


In [19]:
from sklearn.metrics import mean_absolute_error

X = df[['mounth', 'price_kerosin', 'volume']]
y = df['amount']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(df[['mounth', 'price_kerosin', 'volume']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [102150.65300827 844723.93050819   4156.51267919]
Bias: -74052903.70234603
Error: 8381138.276434995


In [21]:
df.amount.median()

65871217.7

Получилось предсказание с абсолютной ошибкой 8 млн рублей, медиана при этом равна 65 млн, ошибка составляет значительную часть выборки 12%.
Попробуем вывести новый признак, который позволит предсказать формулу регрессии с большей точностью
Например, умножим цену на кол-во, получим признак cost - сколько потратили за перевозку.

In [24]:
df['cost'] = df['price_kerosin']*df['volume']
df.head()

,mounth,price_kerosin,volume,amount,cost
0,10,55,10943,28889521.7,601865
1,11,132,10951,69385537.7,1445532
2,7,43,21352,44070529.7,918136
3,10,41,6151,12105169.7,252191
4,11,50,20971,50330401.7,1048550


Теперь попробуем построить модель линейной регрессии на основании этого признака

In [25]:
X = df[['mounth', 'cost']]
y = df['amount']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(df[['mounth', 'cost']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-1.34154676e-10  4.80000000e+01]
Bias: 1.7000000476837158
Error: 3.474019467830658e-08


Ошибка мала, коэффициент сдвига ветра рассчитался корректно. Соответственно, мы сделали правильную сборку 2 признаков в 1, это дало минимализацию ошибки